In [ ]:
#####
# Initial cell for thebe. Not to be run 
#####

# import micropip
# await micropip.install("https://test-files.pythonhosted.org/packages/43/40/324224c95fb3953c1634debaf215d50df579475855323031ae469dfc3540/ipyquizjb-0.2.9-py3-none-any.whl")
%pip install https://test-files.pythonhosted.org/packages/43/40/324224c95fb3953c1634debaf215d50df579475855323031ae469dfc3540/ipyquizjb-0.2.9-py3-none-any.whl

# Test quiz

In [ ]:
import ipywidgets as widgets
from IPython.display import display

display(widgets.HTMLMath(
    value=r"Some math and <i>HTML</i>: \(x^2\) and $$\frac{x+1}{x-1}$$",
    placeholder='Some HTML',
    description='Some HTML',
))

In [ ]:
from ipyquizjb.faceit import display_simple_search

# Test with faceit search (works locally)
# display_simple_search("laplace")

In [ ]:
test_json = r"""{
  "questions": [
    {
      "type": "MULTIPLE_CHOICE",
      "body": "What is the derivative of \\( \\sin(x) \\)?",
      "answers": [
        "\\( -\\cos(x) \\).",
        "\\( \\tan(x) \\).",
        "\\( \\cos(x) \\)."
      ],
      "answer": ["\\( \\cos(x) \\)."],
      "notes": ["The derivative of \\( \\sin(x) \\) is \\( \\cos(x) \\)."]
    },
    {
      "type": "NUMERIC",
      "body": "What's the derivative of z (z'), with respect to x, of z = 4y+x?",
      "answer": "1",
      "notes": []
    }
  ],
  "status": "success"
}"""

# Test without faceit
from ipyquizjb.questions import display_json
display_json(test_json)

In [ ]:
# display(widgets.HTML("<h2>Test your knowledge</h2>"))

# questions: list[Question] = [
#     {
#         "type": "MULTIPLE_CHOICE",
#         "body": "What is your name?",
#         "answers": ["Jakob", "Jørgen", "Ravi"],
#         "answer": ["Jakob"]
#     },
#     {
#         "type": "NUMERIC",
#         "body": "What is 2 + 5?",
#         "answer": "7"
#     },
#     {
#         "type": "NUMERIC",
#         "body": "What is 2/4 + 1/4?",
#         "answer": "0.75"
#     },
#     {
#         "type": "MULTIPLE_CHOICE",
#         "body": "Which of these are cheeses?",
#         "answers": ["Gouda", "Emmentaler", "Brie", "Cat", "Dog"],
#         "answer": ["Gouda", "Emmentaler", "Brie"]
#     }
# ]

# from ipyquiz.questions import display_questions
# display_questions(questions)
from IPython.display import Latex
display(Latex(r"Some math and <i>HTML</i>: \(x^2\) and $$\frac{x+1}{x-1}$$"))




In [ ]:
# Define your function for coding task here:

def test(x, y):
    return 2+x+y

def tet(x, y):
    return x+y

In [ ]:

# display(code_question(
#     "Make a function that returns the sum of two numbers", [((2, 2), 4)]))